In [17]:
import numpy as np
import math
import random
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0
theta_stimato = [0, 0, 0, 0, 0, 0, 0, 0, 0] #stima di theta, start at 0,0,...,0
double_arm = [] #collector of double arm
num_double_arm = 0 #number of double arm
R = [] #collection of reward
Id_arm = [] #id of the double arm at time t associated to the reward
played_superarm = [] #number of times each superarm is played
num_best_super_arm = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0]
num_second_best_super_arm = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0]
final = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0]
sample_correct = [0]*800 #correct at 10, 20, 50, 100, 200, 400, 600, 800
error_reward = [0]*800
error_best_choosen = [0] * 800

delta = 0.05
sigma = 0.1 #noise

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1

def create_union_arm(id_x, id_y):
    res = []
    for i in range(0, K):
        for j in range(0, K):
            res.append(A[id_x].arm_vector[i] * Y[id_y].arm_vector[j])
    return res

def create_all_union():
    for i in range(0, num_arms):
        for j in range (0, num_arms_y):
            curr = create_union_arm(i, j)
            double_arm.append(curr)
            global num_double_arm
            num_double_arm += 1
            played_superarm.append(0)


def calcola_A():
    res = 0
    for i in range(0, t):
        #print("i", i)
        if i == 0:
            #print("i==0, print res", res)
            #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
            res = np.outer(np.transpose(double_arm[Id_arm[i]]), double_arm[Id_arm[i]])
            #print("res", res)
        else:
            #print("i!=0, print i, res", i, res)
            res += np.outer(np.transpose(double_arm[Id_arm[i]]), double_arm[Id_arm[i]])
            #print("A", res)
    res += np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1])
    return res

def calcola_b():
    res = []
    global A
    for i in range(0, t):
        curr = double_arm[Id_arm[i]]
        curr = [R[i] * xx for xx in curr]
        if i == 0:
            res = curr
        else:
            res = [x + y for x, y in zip(res, curr)]
    return res
        
def stima_theta():
    global theta_stimato
    if (t == 0):
        return theta_stimato
    theta_s = np.dot(np.linalg.inv(calcola_A()), calcola_b())
    theta_stimato = theta_s
    return theta_s


def media_reward(index_double_arm):
    return np.dot(double_arm[index_double_arm], np.transpose(stima_theta()))
    
def calcola_bound(index_double_arm):
    ris = 2*sigma*math.sqrt(2)*norm_2_mat(double_arm[index_double_arm], np.linalg.inv(calcola_A()))*math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(t, 2)*K/delta))
    #print("sqrt", math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta)))
    #print("bound", ris)
    return ris
    
def upper_bound(index_double_arm):
    return media_reward(index_double_arm) + calcola_bound(index_double_arm)
    #return self.media_reward(arm_vector)
    
def lower_bound(index_double_arm):
    return media_reward(index_double_arm) - calcola_bound(index_double_arm)
    #return self.media_reward(arm_vector)

class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1        
        
    def generate_double(self):
        for i in range(0, num_arms_y):
            double_arm.append(create_union_arm(self.id, i))

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        played_superarm[self.id * num_arms_y + y_index] += 1
        R.append(reward)
        Id_arm.append(self.id * num_arms_y + y_index)
        return reward
    
    def find_worst_y(self):
        #print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = np.dot(double_arm[self.id * num_arms_y + i], theta_stimato)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
    
    
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)
    
class arm_y():
    arm_vector = np.array([])
    id = 0

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y += 1
        
def run():
    global t
    #print("TIME", t)
    maxi = - 1000
    for i in range(0, num_arms):
        A[i].find_worst_y()
        curr = media_reward(i * num_arms_y + A[i].worst_y)
        #print("curr", curr)
        if curr > maxi:
            best = i
            maxi = curr
        #print("i term and curr value for best, x", i, "y", A[i].worst_y, curr)
    #print("best", best, A[best].worst_y)
    maxi = -1000
    for i in range(0, num_arms):
        if i is not best:
            curr = upper_bound(i * num_arms_y + A[i].worst_y)
            if curr > maxi:
                second_best = i
                maxi = curr
    #print("second best", second_best, A[second_best].worst_y)
    A[best].pull_arm(A[best].worst_y)
    A[second_best].pull_arm(A[second_best].worst_y)
    global num_second_best_super_arm, num_best_super_arm
    num_best_super_arm[best * num_arms_y + A[i].worst_y] +=1
    num_second_best_super_arm[second_best * num_arms_y + A[i].worst_y] +=1
    t+=1
    if (t > 250):
        final[best * num_arms_y + A[i].worst_y] +=1
    return best * num_arms_y + A[i].worst_y

In [18]:
a = arm( [0.6539936932649052, 0.49234274001921385, 0.9073937998343392] )
b = arm( [0.14679488906671767, 0.7429628522780226, 0.5576982329690424] )
c = arm( [0.8930164433455009, 0.8780462992936544, 0.9014394768882377] )
d = arm( [0.2539102367562567, 0.6194135715644297, 0.3121193794381716] )
m = world( [[0.8730240451816763, 0.21159403045606273, 0.28918565354173686], [0.5189626386885673, 0.4849994006831272, 0.9629953198577507], [0.17670422753928094, 0.2532554211216236, 0.9072025363404456]] )
z = arm_y( [0.18050836984570318, 0.6401278349403822, 0.48281685719384904] )
k = arm_y( [0.5890871013415527, 0.26856705274431947, 0.17129466008906424] )
l = arm_y( [0.549236398976063, 0.4311610694683382, 0.9076721344539929] )

In [19]:
best_super = -1
best_value = 0

for i in range(0, num_arms):
    for j in range (0, num_arms_y):
        print("x", i, "y", j, "reward", np.dot(A[i].arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[j].arm_vector))))
        
for i in range(0, num_arms):
    min = 10000
    val_j = 0
    for j in range(0, num_arms_y):
        curr = np.dot(A[i].arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[j].arm_vector)))
        if curr < min:
            min = curr
            val_j = j
    if min > best_value:
        best_value = min
        best_super = i*(j+1)+val_j
print(best_super)
print(best_value)

x 0 y 0 reward 0.6329288594539527
x 0 y 1 reward 0.28028403190475715
x 0 y 2 reward 0.5739501589249083
x 1 y 0 reward 1.930132049218806
x 1 y 1 reward 0.8719451902053754
x 1 y 2 reward 1.7759035558203364
x 2 y 0 reward 0.7813755021007334
x 2 y 1 reward 0.3016634060141854
x 2 y 2 reward 0.7333238007283127
x 3 y 0 reward 1.8584720223370566
x 3 y 1 reward 0.814040901651403
x 3 y 2 reward 1.7293277548503294
4
0.8719451902053754


In [20]:
create_union_arm(0,0)

[0.17774808504992687,
 0.30864476343648256,
 0.36964538166614797,
 0.02118589161378007,
 0.03678753840576917,
 0.04405823550399256,
 0.10365613322688376,
 0.17999025255076986,
 0.21556356524418316]

In [21]:
create_all_union()

In [22]:
double_arm

[[0.17774808504992687,
  0.30864476343648256,
  0.36964538166614797,
  0.02118589161378007,
  0.03678753840576917,
  0.04405823550399256,
  0.10365613322688376,
  0.17999025255076986,
  0.21556356524418316],
 [0.30013823235610865,
  0.19942066560027874,
  0.021176070629647662,
  0.035773640307078365,
  0.023769058360143067,
  0.002523987456964457,
  0.1750295570883226,
  0.11629478357446212,
  0.012349104058094952],
 [0.20791328825077476,
  0.2025181769174174,
  0.3649537583298306,
  0.024781298705458768,
  0.0241382524306104,
  0.04349903834882782,
  0.12124736815312692,
  0.11810113803206945,
  0.21282758339963578],
 [0.3934191565191531,
  0.6831396381070857,
  0.8181555048846147,
  0.1598303386055703,
  0.2775320872516452,
  0.3323835893877046,
  0.30474603152907287,
  0.5291661329745293,
  0.633750642681562],
 [0.6643116868433371,
  0.4413882154112244,
  0.04687010745103621,
  0.2698830498932226,
  0.17931823287372103,
  0.019041434613044517,
  0.5145818318941857,
  0.3419032977458

In [23]:
for i in range (0,num_arms):
    for j in range (0, num_arms_y):
        A[i].pull_arm(j)
        t+=1

In [24]:
R

[0.7432327119719722,
 0.2074943852033741,
 0.5515697213876013,
 2.1227505926231487,
 0.9043724842466027,
 1.7988761848011086,
 0.864560906490626,
 0.21792529688791917,
 0.7939331375819665,
 1.8531725188165358,
 0.7709037997125958,
 1.8304046256190805]

In [25]:
Id_arm

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [26]:
stima_theta()

array([0.2836426 , 0.39447971, 0.54181964, 0.18179071, 0.24344387,
       0.35309955, 0.33639492, 0.47613865, 0.71547203])

In [27]:
for i in range(0,num_arms):
    A[i].find_worst_y()
    print(A[i].worst_y)

1
1
1
1


In [28]:
for i in range(0,num_double_arm):
    print("index", i, "lower_bound", lower_bound(i), "mean", media_reward(i), "upper", upper_bound(i))

index 0 lower_bound 0.45876566835901755 mean 0.6756156084613776 upper 0.8924655485637376
index 1 lower_bound 0.1106449683158639 mean 0.31154090609208107 upper 0.5124368438682982
index 2 lower_bound 0.4128401544399873 mean 0.6116339989742756 upper 0.8104278435085639
index 3 lower_bound 1.401804524048983 mean 1.8462538543745675 upper 2.290703184700152
index 4 lower_bound 0.3910350907031228 mean 0.8492526372988765 upper 1.30747018389463
index 5 lower_bound 1.2623841901075976 mean 1.6721216058996156 upper 2.0818590216916335
index 6 lower_bound 0.4210139468640422 mean 0.7475516092896957 upper 1.0740892717153492
index 7 lower_bound 0.05625177261138581 mean 0.3365837243149762 upper 0.6169156760185666
index 8 lower_bound 0.3798983169962396 mean 0.6781719025227766 upper 0.9764454880493136
index 9 lower_bound 1.2972809261210176 mean 1.692017554679115 upper 2.0867541832372125
index 10 lower_bound 0.363644333358068 mean 0.7727694499827682 upper 1.1818945666074683
index 11 lower_bound 1.16942775960

In [29]:
run()

4

In [30]:
for i in range(0,num_double_arm):
    print("index", i, "lower_bound", lower_bound(i), "mean", media_reward(i), "upper", upper_bound(i))

index 0 lower_bound 0.4565278817726973 mean 0.6741026963625084 upper 0.8916775109523195
index 1 lower_bound 0.12453015897385528 mean 0.3057452803358679 upper 0.4869604016978805
index 2 lower_bound 0.41074592908969615 mean 0.6103105837603655 upper 0.8098752384310348
index 3 lower_bound 1.3972511501511788 mean 1.843107384128446 upper 2.288963618105713
index 4 lower_bound 0.4303526388128597 mean 0.8349829396824148 upper 1.2396132405519698
index 5 lower_bound 1.2580704717626383 mean 1.6693895992373955 upper 2.0807087267121527
index 6 lower_bound 0.41872274853686786 mean 0.7481593189080997 upper 1.0775958892793316
index 7 lower_bound 0.05438913902067849 mean 0.3337099245148968 upper 0.613030710009115
index 8 lower_bound 0.37784099146001726 mean 0.6787512899248561 upper 0.979661588389695
index 9 lower_bound 1.292842369865605 mean 1.6907645899424248 upper 2.0886868100192446
index 10 lower_bound 0.3838942273132074 mean 0.7622278263823367 upper 1.140561425451466
index 11 lower_bound 1.165363847

In [31]:
R

[0.7432327119719722,
 0.2074943852033741,
 0.5515697213876013,
 2.1227505926231487,
 0.9043724842466027,
 1.7988761848011086,
 0.864560906490626,
 0.21792529688791917,
 0.7939331375819665,
 1.8531725188165358,
 0.7709037997125958,
 1.8304046256190805,
 0.788406896205466,
 0.743239441548773]

In [32]:
for i in range (0, 800):
    bst = run()
    if num_best_super_arm.index(max(num_best_super_arm)) == best_super:
        sample_correct[i] += 1
    error_reward[i] += best_value - media_reward(best_super)
    error_best_choosen[i] += best_value - media_reward(bst)

In [33]:
sample_correct

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [34]:
num_second_best_super_arm

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 801, 0]

In [35]:
num_best_super_arm

[0, 0, 0, 0, 801, 0, 0, 0, 0, 0, 0, 0]

In [36]:
for i in range(0,num_double_arm):
    print("index", i, "lower_bound", lower_bound(i), "mean", media_reward(i), "upper", upper_bound(i))

index 0 lower_bound 0.38713943762851116 mean 0.6699988064342659 upper 0.9528581752400207
index 1 lower_bound 0.23567053134956484 mean 0.3113441074556477 upper 0.3870176835617306
index 2 lower_bound 0.34610554494572343 mean 0.6065245431582569 upper 0.8669435413707903
index 3 lower_bound 1.23958236474517 mean 1.8423107871476432 upper 2.4450392095501163
index 4 lower_bound 0.8067000702513393 mean 0.8642120493392232 upper 0.9217240284271072
index 5 lower_bound 1.1108502802502673 mean 1.6683961996078036 upper 2.2259421189653397
index 6 lower_bound 0.34535053398891574 mean 0.7617880535733261 upper 1.1782255731577365
index 7 lower_bound 0.22018665162189038 mean 0.3658492593479327 upper 0.5115118670739751
index 8 lower_bound 0.30878049473806046 mean 0.6908571536829089 upper 1.0729338126277574
index 9 lower_bound 1.1666177072958739 mean 1.7035078905069563 upper 2.2403980737180387
index 10 lower_bound 0.7506743638277344 mean 0.8080725726946383 upper 0.8654707815615421
index 11 lower_bound 1.0468

In [37]:
for i in range(0, num_arms):
    for j in range (0, num_arms_y):
        print("x", i, "y", j, "reward", np.dot(A[i].arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[j].arm_vector))))

x 0 y 0 reward 0.6329288594539527
x 0 y 1 reward 0.28028403190475715
x 0 y 2 reward 0.5739501589249083
x 1 y 0 reward 1.930132049218806
x 1 y 1 reward 0.8719451902053754
x 1 y 2 reward 1.7759035558203364
x 2 y 0 reward 0.7813755021007334
x 2 y 1 reward 0.3016634060141854
x 2 y 2 reward 0.7333238007283127
x 3 y 0 reward 1.8584720223370566
x 3 y 1 reward 0.814040901651403
x 3 y 2 reward 1.7293277548503294


In [38]:
final

[0, 0, 0, 0, 563, 0, 0, 0, 0, 0, 0, 0]

In [39]:
pwd

'C:\\Users\\effel\\Desktop\\LinearBanditExp'

In [40]:
error_reward

[0.039692280733775176,
 0.0420773376320176,
 0.05701578588024048,
 0.07767787132558024,
 0.08191688335441805,
 0.05847734039656871,
 0.03389514666771842,
 0.040308261968697745,
 0.026902145802442545,
 0.01612895961651284,
 0.01814549738891147,
 0.018683159708375308,
 0.019861104293474208,
 0.014991425957523896,
 0.013362938460523388,
 0.016013299134618686,
 0.01478842518054091,
 0.014500603418594049,
 0.010897958859287682,
 0.007415562899895423,
 0.013009172576869288,
 0.016731947776007505,
 0.013598381817993,
 0.02086462750112794,
 0.018176837810415902,
 0.011813445537171119,
 0.010914712394337167,
 0.016320311842734747,
 0.014940877569583177,
 0.017318550986299308,
 0.01668340132588897,
 0.017806619292503267,
 0.016511171335924724,
 0.016092169466564665,
 0.0161540268013336,
 0.014431580730788873,
 0.012768588615439058,
 0.011631103242207286,
 0.013971864921887178,
 0.018648554091138192,
 0.018789833665729283,
 0.020078027363003903,
 0.020352798063446365,
 0.01857382859058876,
 0.018

In [41]:
import numpy as np        
f=open('065014LORO.txt','a')
np.savetxt(f, error_reward, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, error_best_choosen, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, sample_correct, fmt='%1.3f', newline=", ")
f.write("\n")
f.close()

In [42]:
sample_correct

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
